# [MERGED] With 2024-08-30-classifier.ipynb

Keeping this for archival reasons


In [ ]:
import pandas as pd
from functools import reduce
from pathlib import Path

In [ ]:
#! beaker dataset fetch 01J6KF3JRCATRJQ9CPJTRV5VBM
#!beaker dataset fetch 01J6KBM2VCM9EQ7MER26VBXCCM

In [ ]:
LEXICAL_FEATS_PATH = Path("dataset/01J6KF3JRCATRJQ9CPJTRV5VBM/features")
DATASET_PATH = Path(
    "dataset/01J6KBM2VCM9EQ7MER26VBXCCM/helpsteer2_human_vs_gpt4_weighted_for_llama.jsonl"
)

Let's just merge the lexical features


In [ ]:
lexical_features = [
    "rouge",
    "bertscore",
    "bertscore_length",
    "entity_sim",
    "cosine_sim",
    "prompt_len",
    "len_longer",
    "len_shorter",
    "token_len_diff",
]
lexical_feature_files = [
    file
    for file in LEXICAL_FEATS_PATH.glob("*.jsonl")
    if any(file.stem in feat for feat in lexical_features)
]
print(lexical_feature_files)

In [ ]:
lexical_feats_df = reduce(
    lambda left, right: left.merge(
        right, on=["id", "prompt", "completion_a", "completion_b"], how="outer"
    ),
    [pd.read_json(file, lines=True) for file in lexical_feature_files],
)

We don't save the other Bowen's Features, so might need to add that manually


In [ ]:
df = pd.read_json(DATASET_PATH, lines=True).rename(columns={"prompt_hash": "id"})

In [ ]:
finaldf = df.merge(lexical_feats_df, how="left", on="id").drop(
    columns=["prompt", "completion_a", "completion_b"]
)

In [ ]:
finaldf.to_csv("helpsteer2_featureset.csv", index=False)

In [ ]:
finaldf.head(10)